# Importing Libraries

In [1]:
import plotly.offline as pox
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
import alpaca_trade_api as tradeapi
from alpaca_trade_api import TimeFrame, TimeFrameUnit
from sklearn.preprocessing import StandardScaler
from matplotlib.pyplot import figure
from statsmodels.tsa.stattools import adfuller
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
import binascii # Binary to Ascii 
from sklearn.model_selection import train_test_split

# Reading CV File

In [2]:
df=pd.read_csv('Fuzzy_dataset.csv',names=['time', 'ID', 'DLC','Data0','Data1','Data2','Data3','Data4','Data5','Data6','Data7','Label'])
df.dropna(inplace=True)
df

,time,ID,DLC,Data0,Data1,Data2,Data3,Data4,Data5,Data6,Data7,Label
0,1.478196e+09,545,8,d8,0,0,8a,0,0,0,0,R
2,1.478196e+09,2,8,0,0,0,0,0,1,7,15,R
3,1.478196e+09,153,8,0,21,10,ff,0,ff,0,0,R
4,1.478196e+09,130,8,19,80,0,ff,fe,7f,7,60,R
5,1.478196e+09,131,8,17,80,0,0,65,7f,7,9f,R
...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,1.478196e+09,350,8,5,20,a4,68,79,0,0,90,R
1048571,1.478196e+09,02c0,8,14,0,0,0,0,0,0,0,R
1048573,1.478196e+09,316,8,5,22,78,9,22,20,0,6f,R
1048574,1.478196e+09,018f,8,fe,5a,0,0,0,3c,0,0,R


# Data Labelling

In [3]:
# add leading zeros to 4-digit hexadecimal values
df['Label'].replace({'R': 1, 'T': 0}, inplace=True)
df.head()

,time,ID,DLC,Data0,Data1,Data2,Data3,Data4,Data5,Data6,Data7,Label
0,1.478196e+09,545,8,d8,0,0,8a,0,0,0,0,1
2,1.478196e+09,2,8,0,0,0,0,0,1,7,15,1
3,1.478196e+09,153,8,0,21,10,ff,0,ff,0,0,1
4,1.478196e+09,130,8,19,80,0,ff,fe,7f,7,60,1
5,1.478196e+09,131,8,17,80,0,0,65,7f,7,9f,1


# Converting data from hexa to decimal

In [4]:
df['Data0'] = df['Data0'].apply(lambda x: int(x, 16))
df['Data1'] = df['Data1'].apply(lambda x: int(x, 16))
df['Data2'] = df['Data2'].apply(lambda x: int(x, 16))
df['Data3'] = df['Data3'].apply(lambda x: int(x, 16))
df['Data4'] = df['Data4'].apply(lambda x: int(x, 16))
df['Data5'] = df['Data5'].apply(lambda x: int(x, 16))
df['Data6'] = df['Data6'].apply(lambda x: int(x, 16))
df['Data7'] = df['Data7'].apply(lambda x: int(x, 16))
df['ID']    = df['ID'].apply(lambda x: int(x, 16))
y = df[['Label']].copy()
X = df[['ID','DLC','Data0','Data1','Data2','Data3','Data4','Data5','Data6','Data7']].copy()
df.tail(5)

,time,ID,DLC,Data0,Data1,Data2,Data3,Data4,Data5,Data6,Data7,Label
0,1.478196e+09,1349,8,216,0,0,138,0,0,0,0,1
2,1.478196e+09,2,8,0,0,0,0,0,1,7,21,1
3,1.478196e+09,339,8,0,33,16,255,0,255,0,0,1
4,1.478196e+09,304,8,25,128,0,255,254,127,7,96,1
5,1.478196e+09,305,8,23,128,0,0,101,127,7,159,1


In [18]:
a=df[df['Label']==0]
a

,time,ID,DLC,Data0,Data1,Data2,Data3,Data4,Data5,Data6,Data7,Label
1567,1.478196e+09,223,8,140,171,242,38,122,41,26,12,0
1568,1.478196e+09,1770,8,37,16,156,237,91,22,44,24,0
1569,1.478196e+09,765,8,63,189,104,243,195,79,40,212,0
1576,1.478196e+09,301,8,53,69,153,207,9,128,199,119,0
1580,1.478196e+09,826,8,28,4,134,144,127,8,126,201,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1046655,1.478196e+09,351,8,82,150,237,214,250,21,121,210,0
1046656,1.478196e+09,1285,8,10,189,36,29,99,57,150,5,0
1046658,1.478196e+09,1125,8,193,169,57,167,39,39,213,20,0
1046659,1.478196e+09,94,8,209,8,147,0,16,123,162,109,0


In [20]:
a=df[df['Label']==1]
a

,time,ID,DLC,Data0,Data1,Data2,Data3,Data4,Data5,Data6,Data7,Label
0,1.478196e+09,1349,8,216,0,0,138,0,0,0,0,1
2,1.478196e+09,2,8,0,0,0,0,0,1,7,21,1
3,1.478196e+09,339,8,0,33,16,255,0,255,0,0,1
4,1.478196e+09,304,8,25,128,0,255,254,127,7,96,1
5,1.478196e+09,305,8,23,128,0,0,101,127,7,159,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,1.478196e+09,848,8,5,32,164,104,121,0,0,144,1
1048571,1.478196e+09,704,8,20,0,0,0,0,0,0,0,1
1048573,1.478196e+09,790,8,5,34,120,9,34,32,0,111,1
1048574,1.478196e+09,399,8,254,90,0,0,0,60,0,0,1


# Splitting, Training, Testing and Accuracy measurement

In [23]:
# Assuming your data is stored in X (features) and y (labels) variables
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)
# scaler  = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test  = scaler.fit_transform(X_test)
model = KNeighborsClassifier()
# model = LogisticRegression()
# model = DecisionTreeClassifier()
# model = AdaBoostClassifier()
# model = RandomForestClassifier()
# model = SVC()
model.fit(X_train, y_train['Label'])
y_pred = model.predict(X_test)
Accuracy =accuracy_score(y_test, y_pred)
print(Accuracy)

0.999904711753053


# Classification Report

In [25]:
# Classification report
classification_report_data = classification_report(y_test, y_pred)
# Print the Classification report
print(classification_report_data)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     39448
           1       1.00      1.00      1.00    159947

    accuracy                           1.00    199395
   macro avg       1.00      1.00      1.00    199395
weighted avg       1.00      1.00      1.00    199395



In [19]:
model.predict([[223,8,140,171,242,38,122,41,26,12]])

c:\Users\Azaz\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:409: UserWarning:

X does not have valid feature names, but KNeighborsClassifier was fitted with feature names



array([0], dtype=int64)